### Chat bot chats

In this notebook I'm implementing a chatbot chat where I'll use different models to solve a coding challenge. I will initially provide a coding challenge and different models will take turns in coming up with the solution. It's an idea where I could use different models to help me study coding challenges.

##### Model 1: Prompter (Peter)

This model will come up with the prompt and refine the prompts sent to the other models

##### Model 2: Planner (Paul)

This model will come up with the plan on how to solve the problem. It will give 3 suggestions and a recommendation of the best approach.

##### Model 3: Coder (Cody)

This model will write down the code

##### Model 4: Reviewer (Ren)

This model will review the code and give a summary to the user explaining the solution and how we came to the solution.

##### Model 5: Refactor (Rob)

This model will take in the code reviews and refactor the code accordingly


In [ ]:
# Set up the environment

import os

from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

load_dotenv(override=True)
api_key: str | None = os.getenv("OPENROUTER_API_KEY")
base_url: str = "https://openrouter.ai/api/v1"


assert api_key[:8] == "sk-or-v1", "OpenRouter API Key must start with sk-or-v1"

In [ ]:
# Set up the different models


from openai import Stream


openrouter = OpenAI(api_key=api_key, base_url=base_url)

models: list[str] = ["Peter", "Paul", "Cody", "Ren", "Rob"]


def get_user_prompt(conversation: str, name: str) -> str:
    """
    Get the user prompt for the conversation
    """
    other_models_list: list[str] = list(filter(lambda x: x != name, models))
    other_models: str = (
        ", ".join(other_models_list[:-1]) + " and " + other_models_list[-1]
    )
    return f"""
    You are {name}, in conversation with {other_models}.
    The conversation so far is as follows:
    {conversation}
    Now with this, respond with what you would like to say next, as {name}.
    """


def prompter(converstation: str) -> Stream:
    system_prompt = """
    You are Peter who is a senior software engineer. 
    You are responsible for coming up with the prompt for the other models. You will be given an initial 
    prompt and you will come up with clear, concise and specific prompts for the other models. You will ensure that the other
    models understand the task and the constraints. You will also ensure that there are no regressions in as the tasks you will be
    prompting for are coding challenges.
    
    The other models are:
    - Paul: who is good at coming up with plans
    - Cody: who is good at coding
    - Ren: who is good at reviewing code
    - Rob: who is good at refactoring code
    """

    stream = openrouter.chat.completions.create(
        model="openai/gpt-5.1-chat",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(converstation, "Peter")},
        ],
        stream=True,
    )
    return stream


def planner(conversation: str) -> Stream:
    system_prompt = """
    You are Paul who is a senior software engineer. 
    You are responsible for coming up with the plan for the Cody who is a mid level software engineer.
    You will give three suggestions of how to solve the task. You will then give a recommendation of the best approach.
    You will come up with clear, concise and specific plans for the Cody.
    You will ensure that the Cody understands the task and the constraints.
    You will also ensure that there are no regressions in as the tasks you will be planning for are coding challenges.

    The other models are:
    - Peter: who is good at coming up with prompts
    - Cody: who is good at coding
    - Ren: who is good at reviewing code
    - Rob: who is good at refactoring code
    """
    stream = openrouter.chat.completions.create(
        model="google/gemini-3-pro-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(conversation, "Paul")},
        ],
        stream=True,
    )
    return stream


def coder(conversation: str) -> Stream:
    system_prompt = """
    You are Cody who a mid level software engineer. 
    You are responsible for coding the solution to the task. You will be given a plan by Paul and you will code the solution.
    You will ensure that the solution is correct and that it is within the constraints.

    The other models are:
    - Paul: who is good at coming up with plans
    - Peter: who is good at coming up with prompts
    - Ren: who is good at reviewing code
    - Rob: who is good at refactoring code
    """

    stream = openrouter.chat.completions.create(
        model="anthropic/claude-sonnet-4.6",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(conversation, "Cody")},
        ],
        stream=True,
    )
    return stream


def reviewer(conversation: str) -> Stream:
    system_prompt = """
    You are Ren who is a senior QA engineer.
    You are responsible for reviewing all the code.
    You will be given the code and you will review it.
    You will ensure that the code is correct and that it is within the constraints.
    Make sure that the code is DRY, SOLID and follows all the best practices.

    If code needs to refactored, make sure you explicitly add the phrase `refactor the code` to your the response.
    If the code looks good, make sure you explicitly add the phrase `looks good to me` to the response.

    The other models are:
    - Peter: who is good at coming up with prompts
    - Paul: who is good at coming up with plans
    - Cody: who is good at coding
    - Rob: who is good at refactoring code
    """

    stream = openrouter.chat.completions.create(
        model="z-ai/glm-5",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(conversation, "Ren")},
        ],
        stream=True,
    )
    return stream


def refactor(conversation: str) -> Stream:
    system_prompt = """
    You are Rob who is a principal engineer.
    You are responsible for refactoring the code of Cody. You will be given the code and you will refactor it.
    You will ensure that the code is correct and that it is within the constraints.

    The other models are:
    - Paul: who is good at coming up with plans
    - Cody: who is good at coding
    - Ren: who is good at reviewing code
    - Peter: who is good at coming up with prompts
    """

    stream = openrouter.chat.completions.create(
        model="anthropic/claude-sonnet-4.6",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(conversation, "Rob")},
        ],
        stream=True,
    )
    return stream


In [ ]:
# let's start the conversation
from typing import NoReturn

from IPython.display import DisplayHandle, update_display


def display_stream_response(response: Stream, display_handle: DisplayHandle):
    response_stream = ""
    for chunk in response:
        response_stream += chunk.choices[0].delta.content or ""
        update_display(Markdown(response_stream), display_id=display_handle.display_id)
    return response_stream


def begin_converstation(challenge: str) -> NoReturn:
    conversation = f"""
    Initial prompt:

    ## Peter 
     
        Here is the user challenge

    ``` txt
    {challenge}
    ```

    """
    # Display handle
    display_handle = display(Markdown(""), display_id=True)

    # Initial prompt
    prompter_response = display_stream_response(prompter(conversation), display_handle)

    # Planning
    conversation += f"## Peter:\n\n{prompter_response}\n"
    planner_response = display_stream_response(
        planner(prompter_response), display_handle
    )

    # Coding
    conversation += f"## Paul:\n\n{planner_response}\n"
    coder_response = display_stream_response(coder(planner_response), display_handle)

    # Reviewing
    conversation += f"## Cody:\n\n{coder_response}\n"
    reviewer_response = display_stream_response(
        reviewer(coder_response), display_handle
    )
    conversation += f"## Ren:\n\n{reviewer_response}\n"
    refactor_response = display_stream_response(
        refactor(reviewer_response), display_handle
    )
    conversation += f"## Rob:\n\n{refactor_response}\n"
    reviewer_response = display_stream_response(
        reviewer(refactor_response), display_handle
    )

    refactor_count = 1
    # Refactoring
    while refactor_count < 5:
        if "refactor the code" in reviewer_response:
            if refactor_count % 2 == 0:
                conversation += f"## Ren:\n\n{reviewer_response}\n"
                refactor_response = display_stream_response(
                    refactor(reviewer_response), display_handle
                )
                conversation += f"## Rob:\n\n{refactor_response}\n"
                reviewer_response = display_stream_response(
                    reviewer(refactor_response), display_handle
                )
            else:
                conversation += f"## Ren:\n\n{reviewer_response}\n"
                coder_response = display_stream_response(
                    coder(reviewer_response), display_handle
                )
                conversation += f"## Cody:\n\n{coder_response}\n"
                reviewer_response = display_stream_response(
                    reviewer(coder_response), display_handle
                )
        elif "looks good to me" in reviewer_response:
            break

        refactor_count += 1

    # Final response
    conversation += f"### FINAL RESPONSE\n\n{reviewer_response}\n"

    with open("conversation.md", "w", encoding="utf-8") as f:
        f.write(conversation)

    display(Markdown(conversation))


In [ ]:
with open("challenge.md", "r", encoding="utf-8") as f:
    challenge = f.read()

begin_converstation(challenge)